In [1]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from data import *
from model import *

taxi_type = YELLOW
train_sql = getSqlForModeling(taxi_type, isTrain=True)
test_sql = getSqlForModeling(taxi_type, isTrain=False)

scaler = StandardScaler()

model = Sequential([
    Dense(64, input_dim=len(FEATURES), activation="relu"),  # Input layer + first hidden layer
    Dropout(0.2),  # Dropout for regularization
    Dense(32, activation="relu"),  # Second hidden layer
    Dropout(0.2),  # Dropout for regularization
    Dense(16, activation="relu"),  # Third hidden layer
    Dense(1)  # Output layer for regression
])

model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Sample Test
with DR.engn.connect() as conn:
    for df in pd.read_sql(test_sql, conn, chunksize=CHUNK_SIZE):
        sample_X_test = df[FEATURES]
        sample_y_test = df[VARIABLE]
        break

# Training
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

with DR.engn.connect() as conn:
    processed = 0
    for df in pd.read_sql(train_sql, conn, chunksize=CHUNK_SIZE):
        processed += len(df)
        X = df[FEATURES]
        y = df[VARIABLE]

        X_train_scaled = scaler.fit_transform(X)
        X_test_scaled = scaler.transform(sample_X_test)

        model.fit(
            X_train_scaled, y,
            validation_data=(X_test_scaled, sample_y_test),
            epochs=100,
            batch_size=32,
            callbacks=[early_stopping],
            verbose=1
        )
        O.out(f'NN Trained {processed} rows')

# Predictions
with DR.engn.connect() as conn:
    processed = 0
    y_test = []
    y_pred = []
    for df in pd.read_sql(test_sql, conn, chunksize=CHUNK_SIZE):
        processed += len(df)
        X = df[FEATURES]
        y_test.append(df[VARIABLE])

        X_test_scaled = scaler.transform(X)

        y_p = model.predict(X_test_scaled)
        y_pred.append(y_p.flatten())
        O.out(f'Predicted {processed} rows')

# Performance
merged_y_test = pd.concat(y_test, ignore_index=True)
merged_y_pred = np.concatenate(y_pred)

showPerformance(merged_y_test, merged_y_pred)

/Users/njpate/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


sql engine ready
Epoch 1/100
31250/31250 ━━━━━━━━━━━━━━━━━━━━ 30s 951us/step - loss: 12.1733 - mae: 1.4944 - val_loss: 36771.8750 - val_mae: 191.4644
Epoch 2/100
31250/31250 ━━━━━━━━━━━━━━━━━━━━ 30s 954us/step - loss: 5.3793 - mae: 0.8373 - val_loss: 24236.7227 - val_mae: 155.2968
Epoch 3/100
31250/31250 ━━━━━━━━━━━━━━━━━━━━ 29s 932us/step - loss: 4.4661 - mae: 0.7500 - val_loss: 16058.4131 - val_mae: 126.1125
Epoch 4/100
31250/31250 ━━━━━━━━━━━━━━━━━━━━ 30s 945us/step - loss: 4.1275 - mae: 0.7232 - val_loss: 25245.4102 - val_mae: 158.2408
Epoch 5/100
31250/31250 ━━━━━━━━━━━━━━━━━━━━ 30s 946us/step - loss: 3.7457 - mae: 0.6884 - val_loss: 8096.5356 - val_mae: 88.5393
Epoch 6/100
31250/31250 ━━━━━━━━━━━━━━━━━━━━ 29s 933us/step - loss: 3.7028 - mae: 0.6757 - val_loss: 10850.5156 - val_mae: 103.3848
Epoch 7/100
31250/31250 ━━━━━━━━━━━━━━━━━━━━ 30s 952us/step - loss: 3.4615 - mae: 0.6582 - val_loss: 7249.3701 - val_mae: 84.1482
Epoch 8/100
31250/31250 ━━━━━━━━━━━━━━━━━━━━ 30s 951us/step - 

2024-12-01 18:07:43.285884: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: INVALID_ARGUMENT: condition [0], then [32,64], and else [] must be broadcastable
	 [[{{function_node __inference_one_step_on_data_1642}}{{node sequential_1/dropout_1/stateless_dropout/SelectV2}}]]


InvalidArgumentError: Graph execution error:

Detected at node sequential_1/dropout_1/stateless_dropout/SelectV2 defined at (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main

  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 87, in _run_code

  File "/Users/njpate/Library/Python/3.9/lib/python/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/Users/njpate/Library/Python/3.9/lib/python/site-packages/traitlets/config/application.py", line 1046, in launch_instance

  File "/Users/njpate/Library/Python/3.9/lib/python/site-packages/ipykernel/kernelapp.py", line 736, in start

  File "/Users/njpate/Library/Python/3.9/lib/python/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versi